In [2]:
import re
#********************************************************************************
#Видаляємо символи переноса строки спереда із заду
def clearBreakCharacter(expression):
	if expression.startswith('\n') :
		expression = expression[1:] 
		return clearBreakCharacter(expression)
	if expression.endswith('\n'):
		expression = expression[:-1]
		return clearBreakCharacter(expression)
	return expression
#********************************************************************************

#********************************************************************************
# провірка на парність відкритих і закритих дужок 
def isParenthesesBalanced(expression):
    stack = []
    opening_brackets = {'(', '[', '{'}
    closing_brackets = {')': '(', ']': '[', '}': '{'}

    for char in expression:
        if char in opening_brackets:
            stack.append(char)
        elif char in closing_brackets:
            if not stack or stack.pop() != closing_brackets[char]:
                return False
    return not stack  # Все скобки должны быть правильно закрыты
#********************************************************************************

#********************************************************************************
def isExpression(expression):
	# pattern = r'^\s*((([a-zA-Z_]+)([a-zA-Z0-9_]*)\s*=\s*)|\s*\(*\s*(([a-zA-Z_]+)([a-zA-Z0-9_]*)|([0-9]+))\s*\)*\s*[-+*/]\s*)+\s*\(*\s*(([a-zA-Z_]+)([a-zA-Z0-9_]*)|([0-9]+))\s*\)*\s*'
	pattern = r'^\s*(([a-zA-Z_]+\s*=\s*)|\s*\(*\s*(([a-zA-Z_]+)|([0-9]+))\s*\)*\s*[-+*/]\s*)+\s*\(*\s*(([a-zA-Z_]+)|([0-9]+))\s*\)*\s*'
	match = re.fullmatch(pattern, expression)#Тепер в ній провіряємо справність формули
	if match:
		matched_text = match.group()
		if not isParenthesesBalanced(matched_text):#Перевірка на парність закриваючих і відкриваючих дужок
			return False
		else:
			return matched_text
#********************************************************************************

#********************************************************************************
def findCloseBlock(str, index, num):
	indexOpen = str.find('{', index + 1)
	indexClose = str.find('}', index + 1)
	# print(index)
	# print(indexOpen)
	# print(indexClose)
	# Якщо немає парних закриваючих скобок 
	if indexClose == -1:
		raise ValueError("Немає закриваючих скобок")
	# якщо попереду немає відкриваючих вложених скобок або вони є дальше за перші закриваючі  і закриваючі скобки є дальше по індексу як відкиваючі 
	elif (indexOpen == -1 or indexOpen > indexClose) and indexClose > index:
		if num == 0:
			# Повертаємо номер закриваючої дужки
			return indexClose
		else:
			num = num - 1
			return findCloseBlock(str, indexClose, num)

	#Якщо попереду є ще одні відкриваючі скобки 
	elif indexOpen < indexClose:
		num = num + 1
		return findCloseBlock(str, indexOpen, num)
#********************************************************************************

#********************************************************************************
def separateText(list, str):
	str = clearBreakCharacter(str)
	# print(str)
	# print('--------------+++-------------')
	#***************************
	# пошук команд read|write
	match = re.match(r'\s*(read|write).*[\n;]', str)
	if match:
		matched_text = match.group()
		matched_text1= clearBreakCharacter(matched_text)
		# Вырезаем найденный текст из начала строки, учитывая пробелы
		modified_text = str[len(matched_text):]
		list.append(matched_text1)
		# print(modified_text)
		# print('---------------------------')
		return separateText(list, modified_text)

	#***************************
	# пошук команд if|ifnot|while|whilenot
	match = re.match(r'\s*(if|ifnot|while|whilenot).*\n', str)

	if match:
		indexCloseBlock = findCloseBlock(str, str.find('{'), 0)
		modified_text = str[indexCloseBlock+2:]
		# print('***************')
		# print(modified_text)
		# print('***************')
		print('--***--', match)
		comamd = re.match(r'(if|ifnot|while|whilenot)', match.group())
		expression = re.search(r'\[.*\]', match.group() )
		x = re.sub(r'\[|\]','', expression.group())
		x = re.sub(r'\[|\]','', x)
		print(x)
		isExpressionRezult = isExpression(x)
		if isExpressionRezult:
			print('if expression ok')
		else:
			print('if expression error')

		print('----', expression)
		print('----', comamd)

		matched_text = str[:indexCloseBlock+1]
		list.append([matched_text])
		return separateText(list, modified_text)


	#***************************
	# пошук на вирази формула
	# print('***** ',str)
	paternPrev = r'.+;'
	pattern = r'^\s*(([a-zA-Z_]+\s*=\s*)|\s*\(*\s*(([a-zA-Z_]+)|([0-9]+))\s*\)*\s*[-+*/]\s*)+\s*\(*\s*(([a-zA-Z_]+)|([0-9]+))\s*\)*\s*;'
	matchPrev = re.match(paternPrev, str)# Шукаємо строку яка закінчується символом ;
	if matchPrev:
		matchPrev = re.sub(r';','', matchPrev.group())
	# print('11111 ', matchPrev)
	if matchPrev:
		matched_text = isExpression(matchPrev)
		# print('11111 ', matched_text)

		if matched_text:
			list.append(matched_text)
			modified_text = re.sub(pattern, '', str)
			# print('***************+++++++')
			# print(modified_text)
			# print('***************+++++++')
			return separateText(list, modified_text)
	# else:
		# if len(str) > 0:
		# 	print("Sysntasis error")

	return list
#********************************************************************************

program_text = """
read> a;
read> b;
dfdgf = 54654 + drtyrt;
read> c;
ghj = jjk + nj;
if[d*2-3.5]{
	while[b-5]{
		a=a+2*c;
		b=b-1;
	}
}
write> a;"""

lists = separateText([], program_text)
print(len(lists))
print(lists)

for stroka in lists:
	print(stroka)




--***-- <re.Match object; span=(0, 13), match='if[d*2-3.5]{\n'>
7
['read> a;', 'read> b;', 'dfdgf = 54654 + drtyrt', 'read> c;', 'ghj = jjk + nj', ['if[d*2-3.5]{\n\twhile[b-5]{\n\t\ta=a+2*c;\n\t\tb=b-1;\n\t}\n}'], 'write> a;']
read> a;
read> b;
dfdgf = 54654 + drtyrt
read> c;
ghj = jjk + nj
['if[d*2-3.5]{\n\twhile[b-5]{\n\t\ta=a+2*c;\n\t\tb=b-1;\n\t}\n}']
write> a;
